In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging
import logging

import time

class BlogsSpider(scrapy.Spider):
    
    configure_logging(install_root_handler=False)
    
    custom_settings = {
        'DOWNLOAD_DELAY': 1,
        'FEED_URI': './results/blogs_result.json',
        'FEED_FORMAT': 'json'
    }
    
    logging.basicConfig(
        filename='./logs/scrapy_blogs_log.txt',
        format='%(levelname)s: %(message)s',
        level=logging.INFO
    )

    name = "seeking"
    start_urls = [
        'https://seekingalpha.com/latest-articles',
    ]

    def parse(self, response):
        for article in response.xpath("//ul[@class='articles-list']/li"):
            article_link=article.xpath(".//div[@class='media-body']/a[@class='a-title']/@href").extract_first()
            yield response.follow(url=article_link, callback=self.parse_page)
        
        next_page=response.xpath("//li[@class='next']/a/@href").extract_first()
        if next_page is not None:
            time.sleep(30)
            next_page_link= response.urljoin(next_page)
            yield scrapy.Request(url=next_page_link, callback=self.parse)

    def parse_page(self, response):
        yield { 
            'article_name': response.xpath("//h1[@itemprop='headline']/text()").extract_first(),
            'include': response.xpath("//*[@id='a-hd']/div[2]/span[3]/span[2]/@title").extract(),
            'time': response.xpath("//*[@id='a-hd']/div[2]/time/@content").extract_first(),
            'author': response.xpath("//*[@id='author-hd']/div[2]/div[1]/div[1]/a/span/text()").extract_first(),
            'about': response.xpath("//*[@id='about_primary_stocks']").extract(),
            'theme': response.xpath("//*[@class='a-themes']").extract(),
            'summary' : response.xpath("string(//div[@class='a-sum'])").extract(),
            'article' : response.xpath("string(//div[@itemprop='articleBody'])").extract()
        }

In [2]:
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerProcess

settings = get_project_settings()
settings.overrides['FEED_FORMAT'] = 'json'
settings.overrides['FEED_URI'] = './results/blogs_result.json'

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BlogsSpider)
process.start() 

2018-12-29 12:49:57 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-12-29 12:49:57 [scrapy.utils.log] INFO: Versions: lxml 3.8.0.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.9.0, Python 3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Windows-10-10.0.17134-SP0
2018-12-29 12:49:57 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': 1, 'FEED_FORMAT': 'json', 'FEED_URI': './results/blogs_result.json', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2018-12-29 12:49:57 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2018-12-29 12:49:57 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.Ht

2018-12-29 12:50:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230613-antero-resources-shows-recent-natural-gas-price-weakness-overblown> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 12:50:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230613-antero-resources-shows-recent-natural-gas-price-weakness-overblown>
{'article_name': 'Antero Resources Shows Why Recent Natural Gas Price Weakness Is Overblown', 'include': ['Antero Resources Corporation (AR)'], 'time': '2018-12-29T04:37:06Z', 'author': 'KCI Research Ltd.', 'about': ['<span id="about_primary_stocks"><a href="/symbol/AR" title="Antero Resources Corporation" sasource="article_primary_about">Antero Resources Corporation (AR)</a></span>'], 'theme': ['<div class="a-themes"><span id="theme link"><a href="/stock-ideas/long-ideas" sasource="theme_breadcrumb">Long Ideas</a></span><span class="septheme"> | </span><span id="theme link"><a href="/stoc

2018-12-29 12:50:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230614-ross-stores-finally-buy> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 12:50:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230614-ross-stores-finally-buy>
{'article_name': 'Ross Stores Is Finally A Buy Again', 'include': ['Ross Stores, Inc. (ROST)'], 'time': '2018-12-29T04:52:02Z', 'author': 'Josh Arnold', 'about': ['<span id="about_primary_stocks"><a href="/symbol/ROST" title="Ross Stores, Inc." sasource="article_primary_about">Ross Stores, Inc. (ROST)</a></span>'], 'theme': ['<div class="a-themes"><span id="theme link"><a href="/stock-ideas/long-ideas" sasource="theme_breadcrumb">Long Ideas</a></span><span class="septheme"> | </span><span id="theme link"><a href="/stock-ideas/services" sasource="theme_breadcrumb">Services\xa0</a></span></div>'], 'summary': ["Ross Stores has been quite expensive for some time.But recent weak

2018-12-29 12:50:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230615-turning-points-year-end-edition>
{'article_name': 'Turning Points: Year-End Edition', 'include': ['SPY, QQQ, DIA, SH, IWM, TZA, SSO, TNA, VOO, SDS, IVV, SPXU, TQQQ, UPRO, PSQ, SPXL, UWM, RSP, SPXS, SQQQ, QID, DOG, QLD, DXD, UDOW, SDOW, VFINX, URTY, EPS, TWM, SCHX, VV, RWM, DDM, SRTY, VTWO, QQEW, QQQE, FEX, JKD, SPLX, EEH, EQL, SFLA, QQXT, SPUU, IWL, FWDD, SYE, SMLL, SPXE, UDPIX, JHML, OTPIX, RYARX, SPXN, HUSV, RYRSX, SCAP, SPDN, SPXT, SPXV'], 'time': '2018-12-29T04:59:41Z', 'author': 'Hale Stewart', 'about': [], 'theme': ['<div class="a-themes"><span id="theme link"><a href="/market-outlook/todays-market" sasource="theme_breadcrumb">Today\'s Market</a></span><span class="septheme"> | </span><span id="theme link"><a href="/market-outlook" sasource="theme_breadcrumb">Market Outlook</a></span></div>'], 'summary': ['Financial market leading indicators are flashing yellow.Housing is s

2018-12-29 12:50:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230617-chart-week-dreaded-full-frown> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 12:50:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230617-chart-week-dreaded-full-frown>
{'article_name': 'Chart Of The Week: The Dreaded Full Frown', 'include': ['TBT, TLT, TMV, IEF, SHY, TBF, EDV, TMF, PST, TTT, ZROZ, VGLT, TLH, IEI, BIL, TYO, UBT, UST, DLBS, PLW, DTYS, VGSH, SHV, VGIT, GOVT, SCHO, TBX, SCHR, ITE, GSY, TYD, DTYL, EGF, VUSTX, TYBS, DTUS, TUZ, DTUL, DFVL, TAPR, DFVS, TYNS, RISE, FIBR, GBIL, HYDD, UDN, USDU, UUP, RINF'], 'time': '2018-12-29T05:55:04Z', 'author': 'Jeffrey Snider', 'about': [], 'theme': ['<div class="a-themes"><span id="theme link"><a href="/market-outlook/economy" sasource="theme_breadcrumb">Economy</a></span><span class="septheme"> | </span><span id="theme link"><a href="/market-outlook" sasource="theme_breadcrumb">M

...

2018-12-29 13:00:03 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230603-energy-recap-look-back-2018> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 13:00:03 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230603-energy-recap-look-back-2018>
{'article_name': 'Energy Recap: A Look Back At 2018', 'include': ['The United States Oil ETF, LP (USO), UNG, OIL, UGAZ, UWT, TAN, DGAZ, UCO, DWT, SCO, BNO, BOIL, GASL, FCG, DBO, DTO, USL, KOLD, UNL, GASX, DNO, MLPG, OLO, SZO, OLEM, WTIU, OILK, OILX, WTID, USOI, USOU, GAZB, USOD, OILD, OILU, USAI'], 'time': '2018-12-29T00:03:00Z', 'author': 'SA Energy Recap', 'about': ['<span id="about_primary_stocks"><a href="/symbol/USO" title="The United States Oil ETF, LP" sasource="article_primary_about">The United States Oil ETF, LP (USO)</a>, <a href="/symbol/UNG" title="The United States Natural Gas ETF, LP" sasource="article_primary_about">UNG</a>, <a href="/symbol/OIL" titl

2018-12-29 13:00:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230610-bridgeline-digital-inc-blin-roger-kahn-q4-2018-results-earnings-call-transcript> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 13:00:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230610-bridgeline-digital-inc-blin-roger-kahn-q4-2018-results-earnings-call-transcript>
{'article_name': 'Bridgeline Digital, Inc. (BLIN) Roger Kahn on Q4 2018 Results - Earnings Call Transcript', 'include': ['Bridgeline Digital, Inc. (BLIN)'], 'time': '2018-12-29T00:04:03Z', 'author': None, 'about': ['<span id="about_primary_stocks"><a href="/symbol/BLIN" title="Bridgeline Digital, Inc." sasource="article_primary_about_trc">Bridgeline Digital, Inc. (BLIN)</a></span>'], 'theme': ['<div class="a-themes"><span id="transcripts link"><a href="/earnings/earnings-call-transcripts" sasource="theme_breadcrumb">Transcripts</a></span><span class="septheme"> | 

2018-12-29 13:00:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230611-9_2-percent-dividend-yield-earnings-growth-granite-point-merits-closer-look> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 13:00:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230611-9_2-percent-dividend-yield-earnings-growth-granite-point-merits-closer-look>
{'article_name': '9.2% Dividend Yield With Earnings Growth From Granite Point Merits A Closer Look', 'include': ['Granite Point Mortgage Trust (GPMT)'], 'time': '2018-12-29T00:36:18Z', 'author': 'Colorado Wealth Management Fund', 'about': ['<span id="about_primary_stocks"><a href="/symbol/GPMT" title="Granite Point Mortgage Trust" sasource="article_primary_about">Granite Point Mortgage Trust (GPMT)</a></span>'], 'theme': ['<div class="a-themes"><span id="theme link"><a href="/dividends/reits" sasource="theme_breadcrumb">REITs</a></span></div>'], 'summary': ["Shares of GPM

2018-12-29 13:00:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://seekingalpha.com/article/4230612-ncs-multistage-needs-play-cards-right> (referer: https://seekingalpha.com/latest-articles)
2018-12-29 13:00:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://seekingalpha.com/article/4230612-ncs-multistage-needs-play-cards-right>
{'article_name': 'NCS Multistage Needs To Play Its Cards Right', 'include': ['NCS Multistage Holdings (NCSM)'], 'time': '2018-12-29T03:51:53Z', 'author': 'Badsha Chowdhury', 'about': ['<span id="about_primary_stocks"><a href="/symbol/NCSM" title="NCS Multistage Holdings" sasource="article_primary_about">NCS Multistage Holdings (NCSM)</a></span>'], 'theme': ['<div class="a-themes"></div>'], 'summary': ["NCSM's primary concerns are Western Canada-WTI crude price differentials, lower spot prices, and competition from peers.Completions activity slowdown also affects its short-term outlook.NCS Multistage has increased its revenue share from the U.S. i

2018-12-29 13:00:07 [scrapy.core.engine] INFO: Closing spider (finished)
2018-12-29 13:00:07 [scrapy.extensions.feedexport] INFO: Stored json feed (325 items) in: ./results/blogs_result.json
2018-12-29 13:00:07 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 145694,
 'downloader/request_count': 381,
 'downloader/request_method_count/GET': 381,
 'downloader/response_bytes': 45571234,
 'downloader/response_count': 381,
 'downloader/response_status_count/200': 330,
 'downloader/response_status_count/403': 51,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2018, 12, 29, 9, 0, 7, 206832),
 'httperror/response_ignored_count': 51,
 'httperror/response_ignored_status_count/403': 51,
 'item_scraped_count': 325,
 'log_count/DEBUG': 707,
 'log_count/INFO': 69,
 'request_depth_max': 5,
 'response_received_count': 381,
 'scheduler/dequeued': 381,
 'scheduler/dequeued/memory': 381,
 'scheduler/enqueued': 381,
 'scheduler/enqueued/memory': 381,
 'sta